# Level Break Out

### Load the data

In [2]:
# Inital Imports
import pandas as pd
import numpy as np
from scipy import stats
import pandas_ta as ta
import plotly.graph_objects as go
import yfinance as yf


In [3]:
# Specify the stock symbol (NVIDIA in this case) and the date range
stock_symbol = "AAPL"
start_date = "2024-01-11"
end_date = "2024-01-18"

# Fetch stock data from Yahoo Finance
stock_data = yf.download(stock_symbol, start=start_date, end=end_date, interval="1m")

# Create DataFrame from the stock data
df = pd.DataFrame(stock_data)

# Filter rows with non-zero volume
#df = df[df['Volume'] != 0]
df.reset_index(inplace=True)  # Resetting index to get timestamp as a column


[*********************100%%**********************]  1 of 1 completed


In [5]:
# Create function isPivot to detect highs and lows
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, -1 if pivot low, 2 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0

    pivotHigh = 1
    pivotLow = -1
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].Low > df.iloc[i].Low:
            pivotLow=0
        if df.iloc[candle].High < df.iloc[i].High:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 2
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [6]:
# Run isPivot function and store high and low points in new column with a window of +-5 candles 
window=5
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [7]:
# Review Dataframe
df.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume,isPivot
0,2024-01-11 09:30:00-05:00,186.539993,187.050003,186.399994,186.910004,186.910004,1823807,0
1,2024-01-11 09:31:00-05:00,186.914993,187.039902,186.679993,186.830002,186.830002,210564,0
2,2024-01-11 09:32:00-05:00,186.800003,186.904999,186.350006,186.419998,186.419998,291234,0
3,2024-01-11 09:33:00-05:00,186.449997,186.800003,186.350006,186.710007,186.710007,294653,0
4,2024-01-11 09:34:00-05:00,186.710007,186.800003,186.679993,186.759995,186.759995,192722,0


In [8]:
# Create pointpos function to help visualize isPivot
def pointpos(x):
    if x['isPivot']==-1:
        return x['Low']-1e-3
    elif x['isPivot']==1:
        return x['High']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [10]:
# Plot candles and pivot points
dfpl = df
#dfpl = df[df['Date'].dt.year == 2020]
#dfpl = df[(df['Date'].dt.year >= 2020) & (df['Date'].dt.year <= 2023)]
fig = go.Figure(data=[go.Candlestick(x=dfpl['Datetime'],
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl['Datetime'], y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(
    title_text="AAPL Chart with Pivot Points",
    xaxis_title="Date",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False
)
fig.show()

In [11]:
# Drop pointpos columns
df.drop(columns='pointpos', inplace=True)
# Rename isPivot column to signal
df.rename(columns={'isPivot': 'signal'}, inplace=True)
# Review dataframe
df

,Datetime,Open,High,Low,Close,Adj Close,Volume,signal
0,2024-01-11 09:30:00-05:00,186.539993,187.050003,186.399994,186.910004,186.910004,1823807,0
1,2024-01-11 09:31:00-05:00,186.914993,187.039902,186.679993,186.830002,186.830002,210564,0
2,2024-01-11 09:32:00-05:00,186.800003,186.904999,186.350006,186.419998,186.419998,291234,0
3,2024-01-11 09:33:00-05:00,186.449997,186.800003,186.350006,186.710007,186.710007,294653,0
4,2024-01-11 09:34:00-05:00,186.710007,186.800003,186.679993,186.759995,186.759995,192722,0
...,...,...,...,...,...,...,...,...
1555,2024-01-17 15:55:00-05:00,182.779999,182.840103,182.699997,182.820007,182.820007,205123,0
1556,2024-01-17 15:56:00-05:00,182.812393,182.880005,182.779999,182.835007,182.835007,171378,0
1557,2024-01-17 15:57:00-05:00,182.830002,182.849899,182.800003,182.800003,182.800003,178694,0
1558,2024-01-17 15:58:00-05:00,182.804993,182.860001,182.789993,182.804993,182.804993,232892,0


In [12]:
# Export to csv
df.to_csv('signals_appl.csv', index=True)
